In [ ]:
%pip install --upgrade pip

In [ ]:
%pip install kagglehub pandas matplotlib seaborn tensorflow
%pip install --upgrade tensorflow

In [ ]:
%pip install numpy h5py pyyaml protobuf transformers

In [ ]:
%pip install transformers

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Download latest version
path = kagglehub.dataset_download("uciml/sms-spam-collection-dataset")

print("Path to dataset files:", path)

C:\Users\Алексей\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|████████████████████████████████████████████████████████████████████████████████| 211k/211k [00:00<00:00, 611kB/s]

Extracting files...
Path to dataset files: C:\Users\Алексей\.cache\kagglehub\datasets\uciml\sms-spam-collection-dataset\versions\1


In [33]:
import pandas as pd

data = pd.read_csv('spam.csv', encoding='ISO-8859-1')

data = data[['v1', 'v2']]

print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None


,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [34]:
missing_values = data.isnull().sum()
print("Пропущенные значения:\n", missing_values[missing_values > 0])

Пропущенные значения:
 Series([], dtype: int64)


In [35]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)

X = data['v2'].values.reshape(-1, 1)
y = data['v1']                        

X_resampled, y_resampled = ros.fit_resample(X, y)

balanced_data = pd.DataFrame(X_resampled, columns=['v2'])
balanced_data['v1'] = y_resampled

print("Class counts after oversampling:\n", balanced_data['v1'].value_counts())

Class counts after oversampling:
 v1
ham     4825
spam    4825
Name: count, dtype: int64


In [36]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

data['v1'] = data['v1'].astype(str)

data['v1'] = data['v1'].map({'ham': 0, 'spam': 1})

data = data.dropna(subset=['v1', 'v2'])

X_text = data['v2'].values.tolist()
y = data['v1'].values

In [39]:
X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text, y, test_size=0.2, random_state=42
)

In [40]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_text)

X_train_seq = tokenizer.texts_to_sequences(X_train_text)
X_test_seq = tokenizer.texts_to_sequences(X_test_text)

max_length = 200
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_padded)
X_test_scaled = scaler.transform(X_test_padded)

model = RandomForestClassifier()
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)
y_prob = model.predict_proba(X_test_scaled)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96       965
           1       0.98      0.53      0.68       150

    accuracy                           0.93      1115
   macro avg       0.95      0.76      0.82      1115
weighted avg       0.94      0.93      0.93      1115

ROC-AUC: 0.928566493955095
